In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = pd.read_csv("Resources/Clean/aggregated_review_scores.csv")

In [2]:
df_scaled = df.copy()

In [3]:
df.look = df.look.round(2)
df.smell = df.smell.round(2)
df.taste = df.taste.round(2)
df.feel = df.feel.round(2)

In [4]:
# select features for the model
features = ['estimated_ibu', 'look', 'smell', 'taste', 'feel', 'abv']

# scale the features to a range between 0 and 1
scaler = MinMaxScaler()
df_scaled[features] = scaler.fit_transform(df_scaled[features])

# create a dataframe with only the relevant features
df_model = df_scaled[['beer_id', 'beer_name', 'beer_style'] + features + ['brewery_name', 'city', 'state']]
df = df[['beer_name', 'beer_style'] + features + ['availability', 'brewery_name', 'city', 'state', 'types']]

In [5]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 10

# initialize the model with the number of neighbors
model = NearestNeighbors(n_neighbors=k)

# fit the model to the data
model.fit(df_model[features])

# define a function to recommend beers based on a given beer name
def recommend_beers_name(beer_name):
    # get the beer_id of the given beer name
    beer_id = df_model[df_model['beer_name'] == beer_name]['beer_id'].iloc[0]
    
    # get the index of the beer in the model dataframe
    idx = df_model[df_model['beer_id'] == beer_id].index[0]
    
    # get the features of the beer
    beer_features = df_model.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(beer_features)
    
    # get the beer names of the nearest neighbors
    beer_names = df.iloc[indices[0], :][1:]
    
    return beer_names

In [6]:
recommend_beers_name('700c Wit')

C:\Users\Gage\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,beer_name,beer_style,estimated_ibu,look,smell,taste,feel,abv,availability,brewery_name,city,state,types
1695,Cornerstone,American Cream Ale,16,3.25,3.27,3.27,3.31,5.4,Year-round,Solid Rock Brewing,Spicewood,TX,Brewery
2198,The Badge,American Blonde Ale,20,3.26,3.19,3.25,3.19,4.9,Rotating,Nine-Band Brewing Company,Allen,TX,"Brewery, Bar"
1585,No Hitter,American Cream Ale,16,3.19,3.25,3.31,3.19,4.2,Year-round,Idle Vine Brewing Co,Pflugerville,TX,"Brewery, Bar"
2179,Spring Break,Belgian Witbier,15,3.12,3.12,3.06,3.06,5.2,Spring,Audacity Brewhouse,Denton,TX,"Brewery, Bar, Eatery"
1893,Hung Jury Hefeweizen,German Hefeweizen,12,3.48,3.22,3.22,3.18,5.0,Year-round,Legal Draft Beer Company,Arlington,TX,"Brewery, Bar, Beer-to-go"
1473,Berry White,Belgian Witbier,15,3.33,3.25,2.92,3.08,4.4,Rotating,Whole Foods - Post Oak,Houston,TX,"Brewery, Bar, Eatery, Beer-to-go"
844,Tejas Clara,American Light Lager,10,3.17,3.17,3.04,3.17,4.3,Rotating,Big Bend Brewing Co.,Alpine,TX,Brewery
10,Boomtown Blonde,American Blonde Ale,20,3.24,3.03,3.15,3.20,5.5,Year-round,Spindletap Brewery,Houston,TX,"Brewery, Bar"
1434,GRITz,American Cream Ale,16,3.33,3.19,3.30,3.34,4.4,Year-round,Braindead Brewing,Dallas,TX,"Brewery, Bar, Eatery"


In [7]:
def recommend_beers_style(beer_style):
    # get the beer_ids of the given beer style
    beer_ids = df_model[df_model['beer_style'] == beer_style]['beer_id']
    
    # get the indices of the beers in the model dataframe
    indices = [df_model[df_model['beer_id'] == beer_id].index[0] for beer_id in beer_ids]
    
    # get the features of the beers
    beer_features = df_model.loc[indices, features]
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(beer_features)
    
    # get the beer names of the nearest neighbors
    beer_names = df.loc[indices[0]]
    
    return beer_names

In [8]:
recommend_beers_style("American Imperial Stout")

,beer_name,beer_style,estimated_ibu,look,smell,taste,feel,abv,availability,brewery_name,city,state,types
43,Buffalo Bayou Black Raz,American Imperial Stout,65,3.93,3.94,3.91,3.85,8.00,Rotating,Buffalo Bayou Brewing Co,Houston,TX,"Brewery, Bar"
963,Buried Hatchet Stout,American Imperial Stout,65,3.88,3.96,4.00,3.85,8.25,Year-round,Southern Star Brewing Company,Conroe,TX,Brewery
1513,Goesting,Belgian IPA,65,3.83,3.83,3.92,3.92,7.40,Rotating,Whole Foods - Post Oak,Houston,TX,"Brewery, Bar, Eatery, Beer-to-go"
946,Obvious Equation,American IPA,60,3.95,4.00,3.90,3.85,7.95,Rotating,Ingenious Brewing Company,Humble,TX,"Brewery, Bar, Beer-to-go"
2117,The Giant,American Imperial Stout,65,4.05,3.98,3.85,3.98,8.00,Winter,"Fredericksburg Brewing Company, Inc.",Fredericksburg,TX,"Brewery, Bar, Eatery, Beer-to-go"
318,Wet Hop IPA,American IPA,60,3.95,3.90,3.95,3.85,7.80,Rotating,Pinthouse Pizza Craft Brewpub,Austin,TX,"Brewery, Bar, Eatery, Beer-to-go"
1381,More Better Oranges & Cream Milkshake,New England IPA,60,3.92,4.00,3.92,3.83,8.30,Rotating,Red Horn Coffee House & Brewing Co.,Cedar Park,TX,"Brewery, Bar, Eatery"
1442,Anodyne,American Wheatwine Ale,65,3.89,3.87,3.90,3.86,9.00,Rotating,Revolver Brewing,Granbury,TX,Brewery
1268,Greenhouse IPA,American IPA,60,3.92,3.92,3.93,3.88,7.60,Rotating,Hops and Grain Brewery,Austin,TX,"Brewery, Bar"
404,Battle Axe,American Imperial Red Ale,70,3.89,3.84,3.96,3.83,8.00,Fall,Austin Beerworks,Austin,TX,"Brewery, Bar"
